In [3]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build, Resource
from datetime import date, timedelta
from typing import Dict
import numpy as np
from polyfuzz.models import TFIDF
from polyfuzz import PolyFuzz

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Variablen anpassen

Welche Domain soll analysiert werden?

In [5]:
############################################################################################
#Variables 
### Zunächst wird danach gefiltert, welche Suchbegriffe nicht enthalten sein sollen (notContains)
brand = "guru"

#AT(aut), Switzerland(che), Netherlands(nl), Spain(esp), Germany(deu)
COUNTRY_FILTER = ["deu"]

#DE(urlaubsguru.de), NLD(holidayguru.nl), ES(holidayguru.es), AT(urlaubsguru.at). CH(holidayguru.ch)
domain_name = "urlaubsguru.de" 

#Hier wird der Dateipfad des ScreamingFrog Crawls angegeben - benötigt "internal_html.csv"
frog_crawl_path = "/Users/paulherzog/Downloads/internal_html (3).csv"

#wie viele monate an daten möchtest du haben?
months_wanted = 3

#Welcher Threshold soll angewandt werden? 50% heißt z.B.: Alle Title / H1 Tags, die zu weniger als 50% mit der Top Query übereinstimmen, werden als 'Quick Win' getagged
threshold_wanted = 80

#Wo soll das finale Excel Sheet mit den Daten gespeichert werden?
export_path = "/Users/paulherzog/Downloads/"
############################################################################################

## Data Pipeline GSC

In diesem Bereich werden die Daten der letzten 30 Tage aus der Google Search Console gezogen.

In [6]:
#Variablen für den API Request
DIMENSIONS_BYURL = ["page", "query", "date", "country"]
DOMAIN = "sc-domain:" + domain_name
credential_filepath = ".secrets/creds.json"

#berechnung der zeit
months_in_days = int(months_wanted * 7 * 4.33)
delta_days = months_in_days
end_date = date.today()
start_date = end_date - timedelta(days=delta_days)
#variable für spätere bezeichnung der column
months_in_days_str = str(months_in_days)
click_column_name = "clicks_last_" + months_in_days_str + "_days_for_url"
avg_position_column_name = "avg_position _" + months_in_days_str + "_for_top_query"
#variable für threshold setzen
threshold_wanted_float = threshold_wanted / 100

In [8]:
#API Request GSC
API_SERVICE_NAME = "webmasters"
API_VERSION = "v3"
SCOPE = ["https://www.googleapis.com/auth/webmasters.readonly"]
MAX_ROWS = 25_000

def auth_using_key_file(key_filepath):
    credentials = service_account.Credentials.from_service_account_file(
        key_filepath, scopes=SCOPE
    )
    service = build(API_SERVICE_NAME, API_VERSION, credentials=credentials)
    return service
def query(client: Resource, payload: Dict[str, str]) -> Dict[str, any]:
    response = client.searchanalytics().query(siteUrl=DOMAIN, body=payload).execute()
    return response

KEY_FILE = credential_filepath
service = auth_using_key_file(key_filepath=KEY_FILE)

i = 0
reponse_by_url = []
while True:
    payload_main_range = {
        "startDate": start_date.strftime("%Y-%m-%d"),
        "endDate": end_date.strftime("%Y-%m-%d"),
        "dimensions": DIMENSIONS_BYURL,
        "dimensionFilterGroups": [{
            "filters": [{
                "dimension": "country",
                "operator": "contains",
                "expression": COUNTRY_FILTER
                },
            {
                "dimension": "query",
                "operator": "notContains",
                "expression": brand
            },
            {
                "dimension": "page",
                "operator": "notContains",
                "expression": "#"
            }]
        }],
        "rowLimit": MAX_ROWS,
        "startRow": i * MAX_ROWS
    }

    # make request to API
    response_main_range = query(service, payload_main_range)

    # if there are rows in the response, append to the temporary list
    if response_main_range.get("rows"):
        reponse_by_url.extend(response_main_range["rows"])
        i += 1
    else:
        break

    print(f"Collected {len(reponse_by_url):,} rows.")
    
# Create a DataFrame from the temporary list
by_url_data = pd.DataFrame(reponse_by_url)
by_url_data[DIMENSIONS_BYURL] = pd.DataFrame(by_url_data["keys"].tolist(), index=by_url_data.index)
df_raw_data = by_url_data.drop(columns="keys")

Collected 25,000 rows.
Collected 50,000 rows.
Collected 75,000 rows.
Collected 100,000 rows.
Collected 125,000 rows.
Collected 150,000 rows.
Collected 175,000 rows.
Collected 200,000 rows.
Collected 225,000 rows.
Collected 250,000 rows.
Collected 275,000 rows.
Collected 300,000 rows.
Collected 325,000 rows.
Collected 350,000 rows.
Collected 375,000 rows.
Collected 400,000 rows.
Collected 425,000 rows.
Collected 450,000 rows.
Collected 475,000 rows.
Collected 500,000 rows.
Collected 525,000 rows.
Collected 550,000 rows.
Collected 575,000 rows.
Collected 600,000 rows.
Collected 625,000 rows.
Collected 650,000 rows.
Collected 675,000 rows.
Collected 700,000 rows.
Collected 725,000 rows.
Collected 750,000 rows.
Collected 775,000 rows.
Collected 800,000 rows.
Collected 825,000 rows.
Collected 850,000 rows.
Collected 875,000 rows.
Collected 900,000 rows.
Collected 925,000 rows.
Collected 950,000 rows.
Collected 975,000 rows.
Collected 1,000,000 rows.
Collected 1,025,000 rows.
Collected 1,050

## Laden aktueller Crawling Daten

- Benötigt die Datei "internal_html.csv" aus einem aktuellen ScreamingFrog Crawl

In [9]:
frog_crawl = pd.read_csv(frog_crawl_path, low_memory=False)

## Data Manipulation

Aufbereitung der Daten mit dem Ziel:
- Top Keyword pro URL nach Clicks zu finden

In [10]:
#rohdaten filtern - alle queries ohne Click werden rausgehaut
filtered_df = df_raw_data[df_raw_data["clicks"] > 0]

#Datum zu einem date data type umwandeln
filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y-%m-%d")

#Hier wird eine Liste aller einzigartiger URLs erstellt
list_of_unique_urls = filtered_df.groupby("page")["clicks"].sum().sort_values(ascending=False).index.to_list()

#dataframe mit der Summe an Klicks pro URL - wird später benötigt für einen Join
pivot_with_performance_data = filtered_df.groupby("page")["clicks"].sum().sort_values(ascending=False)




/var/folders/rw/4s6mkb8x60s8x4flgt9mbjzr0000gn/T/ipykernel_49964/3033094519.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y-%m-%d")


In [11]:
#Hier wird durch jede einzelne dieser URLs gelooped, die Rohdaten pro URL gefiltert und anschließend das Top Keyword pro URL nach Clicks gezogen
list_with_urls_and_top_queries = []

for url in list_of_unique_urls:
    filtered_df_for_current_url = filtered_df[filtered_df["page"] == url]
    top_kw = filtered_df_for_current_url.groupby(by="query")["clicks"].sum().sort_values(ascending=False).head(1).index.tolist()    
    top_kw_str = ', '.join(top_kw)
    list_with_urls_and_top_queries.append((url, top_kw_str))

#Erstellen eines neuen Dataframes mit zwei spalten: URL und Top Query
df_with_urls_and_top_queries = pd.DataFrame(list_with_urls_and_top_queries, columns=("url", "top_query"))


In [12]:
#Hier werden die Frog Daten mit den GSC Daten gejoined
df_joined_gsc_and_frog = df_with_urls_and_top_queries.merge(frog_crawl, left_on='url', right_on='Address', how='inner')[['url', 'top_query', 'Title 1', 'H1-1']]
# und anschließend Title und H1 auf lowercase gestellt, damit die Similarity Berechnung anschließend keine Probleme macht
df_joined_gsc_and_frog['Title 1'] = df_joined_gsc_and_frog['Title 1'].str.lower()
df_joined_gsc_and_frog['H1-1'] = df_joined_gsc_and_frog['H1-1'].str.lower()
#hier wird noch der name der column umbenannt
df_joined_gsc_and_frog = df_joined_gsc_and_frog.rename(columns={'Title 1': 'title_1'})
df_joined_gsc_and_frog = df_joined_gsc_and_frog.rename(columns={'H1-1': 'h1'})
#remove empty rows, where either title_1 or h1 is missing
df_joined_gsc_and_frog = df_joined_gsc_and_frog.dropna(subset=['title_1', 'h1'])

#join mit performance daten um anschließend duplicate top queries rauszuhauen, basierend auf clicks
df_with_performance_data = pd.DataFrame(pivot_with_performance_data).reset_index()
df_joined_gsc_and_frog_without_duplicates = df_with_performance_data.merge(df_joined_gsc_and_frog, left_on = "page", right_on = "url", how = "inner")
df_joined_gsc_and_frog_without_duplicates = df_joined_gsc_and_frog_without_duplicates.sort_values(by='clicks', ascending=False).drop_duplicates(subset='top_query', keep='first')

#pivot table mit Avg. Ranking pro Top KW pro URL
list_with_top_queries_and_ranking = []

for row in df_joined_gsc_and_frog_without_duplicates.itertuples():
    url = row.page
    top_kw_for_url = row.top_query
    filtered_df_for_avg_pos = filtered_df[(filtered_df["page"] == url) & (filtered_df["query"] == top_kw_for_url)]
    avg_pos_for_top_query = filtered_df_for_avg_pos.groupby("page")["position"].mean()
    avg_pos_for_top_query = avg_pos_for_top_query[0].round(3)
    list_with_top_queries_and_ranking.append((top_kw_for_url, avg_pos_for_top_query))

#Neues Dataframe erstellen mit Top Query und Position
df_with_top_queries_and_position = pd.DataFrame(list_with_top_queries_and_ranking, columns=("top_query", avg_position_column_name))


In [13]:
#Top Queries, Title Tags und H1 Tags werden als Listen abgespeichert
top_query_list = df_joined_gsc_and_frog_without_duplicates["top_query"].tolist()
title_tag_list = df_joined_gsc_and_frog_without_duplicates["title_1"].tolist()
h1_list = df_joined_gsc_and_frog_without_duplicates["h1"].tolist()

#Hier wird PolyFuzz verwendet um die Ähnlichkeit zwischen Top Query und Title / H1 zu berechnen (Source: https://maartengr.github.io/PolyFuzz/api/models/distance/)
model = PolyFuzz("EditDistance")
#Function
def sim_calc(top_query_list, sim_wanted_list, column_name_for_from, column_name_for_to, column_name_for_sim):
        #hier werden pairs erstellt zwischen top query und title/h1 - damit immer die korrekte Kombination miteinander verglichen wird
        pairs = list(zip(top_query_list, sim_wanted_list))
        similarities = []
        #hier wird durch alle pairs durchgelooped und die Similarity berechnet
        for pair in pairs:
            model.match([pair[0]], [pair[1]])
            similarity = model.get_matches()
            similarity_float = similarity["Similarity"][0]
            similarities.append(similarity_float)

        outcome = pd.DataFrame({
              column_name_for_from: top_query_list,
              column_name_for_to: sim_wanted_list,
              column_name_for_sim: similarities
        })

        outcome.sort_values(column_name_for_sim, ascending=False, inplace=True)
        return outcome

df_top_query_title_sim = sim_calc(top_query_list, title_tag_list, "top_query", "title_tag", "sim_topq_to_title")
df_top_query_h1_sim = sim_calc(top_query_list, h1_list, "top_query", "h1_tag", "sim_topq_to_h1")

#Anschließend werden beide Tabellen miteinander gejoined
df_joined_title_h1_similarities = df_top_query_title_sim.merge(df_top_query_h1_sim, left_on = "top_query", right_on = "top_query", how = "inner")
#...und noch um die URL ergänzt
df_with_final_similarities_and_joined_url = df_joined_title_h1_similarities.merge(df_with_urls_and_top_queries, left_on = "top_query", right_on = "top_query", how = "inner")
#...und noch Ranking ergänzt
df_with_final_similarities_and_joined_url = df_with_final_similarities_and_joined_url.merge(df_with_top_queries_and_position, on = "top_query", how = "inner")

In [14]:
#hier werden jetzt noch GSC Daten hinzugezogen
final_df_with_gsc_data = df_with_final_similarities_and_joined_url.merge(df_joined_gsc_and_frog_without_duplicates, left_on = "url", right_on = "url", how = "inner", suffixes=('_first', '_second'))
final_df_with_gsc_data = final_df_with_gsc_data.rename(columns={
    "clicks": click_column_name
})
final_df = final_df_with_gsc_data.sort_values(by=click_column_name, ascending=False)

In [15]:
#Hier wird aufgeräumt - Kolumnen entfernt und jene mit Suffix umbenannt
columns_to_drop = [col for col in final_df.columns if col.endswith('_second')]
columns_to_rename = {col: col.replace('_first', '') for col in final_df.columns if col.endswith('_first')}
final_df = final_df.drop(columns=columns_to_drop)
final_df = final_df.rename(columns=columns_to_rename)
final_df = final_df.drop(columns=["page", "title_1", "h1"])
#anschließend kolumnen neu ordnen
desired_columns_order = ['url', 'top_query', 'title_tag', 'sim_topq_to_title', 'h1_tag', 'sim_topq_to_h1', click_column_name, avg_position_column_name]
final_df = final_df[desired_columns_order]
final_df = final_df.reset_index(drop=True)

## Finalisierung & Kategorisierung der Daten

Hier werden die Daten noch getagged und unterteilt in:
- Chancen für Title Tag Update
    - URLs, bei denen Top Query und Title Tag weniger als x% übereinstimmt
- Chancen für H1 Tag Update
    - URLs, bei denen Top Query und H1 Tag weniger als x% übereinstimmt

Gleichzeitig ist eine Chance nur dann eine, wenn die aktuelle Position für diese Top-Query folgende Anforderungen erfüllt:
- Position ist schlechter/gleich wie 2 (ranking >= 2)

Beide Tags sind sortiert nach Klicks.

In [16]:
#Hier wird die Tagging Funktion erstellt
def tagging(row, column_to_test):
    value = row[column_to_test]
    avg_position = row[avg_position_column_name]
    if isinstance(value, (int, float)) and value <= threshold_wanted_float and avg_position >= 2:
        return 'Chance'

df = final_df
#tagging basierend auf 
df['tag_for_title'] = df.apply(lambda row: tagging(row, 'sim_topq_to_title'), axis=1)
df['tag_for_h1'] = df.apply(lambda row: tagging(row, 'sim_topq_to_h1'), axis=1)

In [17]:
df_only_title_chances = df[df["tag_for_title"] == "Chance"].drop(columns=["h1_tag", "sim_topq_to_h1", "tag_for_title", "tag_for_h1"]).reset_index(drop=True)
df_only_h1_chances = df[df["tag_for_h1"] == "Chance"].drop(columns=["title_tag", "sim_topq_to_title", "tag_for_title", "tag_for_h1"]).reset_index(drop=True)

In [18]:
print("Quick-Win-Übersicht für:")
print(f"Domain: {domain_name}")
print(f"Threshold: {threshold_wanted}%")
print(f"Timerange: {start_date} - {end_date}. Dies entspricht {months_wanted} Monat(e) bzw. {months_in_days} Tage.)")
print("-----------------------------------------------------------------------------")
print(f"Aktuell gibt es {len(df_only_title_chances)} Chancen für einen Title-Tag-Quick-Win.")
print(f"Aktuell gibt es {len(df_only_h1_chances)} Chancen für einen H1-Tag-Quick-Win.")
print("-----------------------------------------------------------------------------")

Quick-Win-Übersicht für:
Domain: urlaubsguru.de
Threshold: 80%
Timerange: 2023-10-07 - 2024-01-05. Dies entspricht 3 Monat(e) bzw. 90 Tage.)
-----------------------------------------------------------------------------
Aktuell gibt es 317 Chancen für einen Title-Tag-Quick-Win.
Aktuell gibt es 441 Chancen für einen H1-Tag-Quick-Win.
-----------------------------------------------------------------------------


In [19]:
df_only_title_chances.head(25)

,url,top_query,title_tag,sim_topq_to_title,clicks_last_90_days_for_url,avg_position _90_for_top_query
0,https://www.urlaubsguru.de/reisemagazin/reisewarnung-aegypten-sicherheit/,reisewarnung ägypten? - hurghada aktuell,reisewarnung ägypten ➡ aktuelles zur sicherheit,0.698851,25341,3.933
1,https://www.urlaubsguru.de/,urlaub,urlaub günstig buchen mit urlaubsguru: findet euren traumurlaub,0.600000,11106,9.115
2,https://www.urlaubsguru.de/reisekalender/silvester/,silvester 2023,silvesterreisen angebote 2024 / 2025 - dein unvergesslicher jahreswechsel,0.704348,10696,17.597
3,https://www.urlaubsguru.de/reisemagazin/colmar-im-elsass/,colmar,top 6 sehenswürdigkeiten in colmar - hübsche stadt mit viel charme,0.600000,3155,10.341
4,https://www.urlaubsguru.de/lexikon/buchungsklassen/,buchungsklassen lufthansa,was sind buchungsklassen? | urlaubsguru lexikon,0.676047,2785,3.618
5,https://www.urlaubsguru.de/reisemagazin/skurile-und-lustige-ortsnamen/,lustige städtenamen,die skurrilsten und witzigsten ortsnamen der welt | urlaubsguru,0.521053,2456,2.511
6,https://www.urlaubsguru.de/reisemagazin/bordingpass-postkarte/,flugticket vorlage zum ausdrucken pdf,boardingpass postkarte zum ausdrucken | urlaubsguru,0.521569,2390,2.011
7,https://www.urlaubsguru.de/reisemagazin/santa-claus-village-finnisch-lappland/,weihnachtsdorf finnland,das santa claus village in finnisch lappland,0.480000,2346,4.885
8,https://www.urlaubsguru.de/reisemagazin/goa-indien/,goa,goa - das paradies für indien-anfänger | urlaubsguru,0.600000,2268,7.043
9,https://www.urlaubsguru.de/reisemagazin/korfu-tipps/,korfu,die top 6 sehenswürdigkeiten für korfu inkl. geheimtipps ☀️,0.600000,2247,11.865


In [20]:
df_only_h1_chances.head(25)

,url,top_query,h1_tag,sim_topq_to_h1,clicks_last_90_days_for_url,avg_position _90_for_top_query
0,https://www.urlaubsguru.de/reisekalender/silvester/,silvester 2023,silvesterreisen 2024 / 2025 – euer traumurlaub zum jahreswechsel,0.704348,10696,17.597
1,https://www.urlaubsguru.de/reisekalender/beste-reisezeit-dubai/,wetter dubai januar,beste reisezeit dubai,0.500000,9323,3.079
2,https://www.urlaubsguru.de/reisekalender/beste-reisezeit-dominikanische-republik/,klimatabelle dominikanische republik,beste reisezeit dominikanische republik,0.740678,5419,2.765
3,https://www.urlaubsguru.de/freizeitaktivitaeten/thermen-und-badewelt-sinsheim/,sinsheim therme preise,thermen & badewelt sinsheim,0.697959,5374,4.351
4,https://www.urlaubsguru.de/reisekalender/reiseziele-oktober/,wo ist es im oktober warm,reiseziele im oktober 2024,0.588235,5055,4.295
5,https://www.urlaubsguru.de/reisemagazin/essen-flugzeug-erlaubt/,darf man essen mit ins flugzeug nehmen,welches essen darf ins flugzeug?,0.732857,4683,2.955
6,https://www.urlaubsguru.de/kurzreisen/,kurztrip,kurzurlaub 2024,0.642857,4431,6.393
7,https://www.urlaubsguru.de/reisekalender/beste-reisezeit-gran-canaria/,gran canaria wetter,beste reisezeit gran canaria,0.735484,4341,11.393
8,https://www.urlaubsguru.de/reisekalender/beste-reisezeit-fuerteventura/,fuerteventura wetter,beste reisezeit fuerteventura,0.748485,4068,13.330
9,https://www.urlaubsguru.de/reisekalender/beste-reisezeit-teneriffa/,klimatabelle teneriffa,beste reisezeit teneriffa,0.553191,3993,3.261


## Export der Daten als Excel

In [ ]:
#Hier wird der Pfad und der Dateiname gebaut
domain_name_for_excel_name = domain_name.replace(".", "_")
excel_file_name = export_path + end_date.strftime("%Y-%m-%d") + "-" + domain_name_for_excel_name + "_title_h1_quickwins.xlsx"

In [ ]:
with pd.ExcelWriter(excel_file_name) as writer:
    df_only_title_chances.to_excel(writer, sheet_name = "Title Tag Quick Wins")
    df_only_h1_chances.to_excel(writer, sheet_name = "H1 Tag Quick Wins")
    df.to_excel(writer, sheet_name="All data")